<a href="https://www.kaggle.com/code/quangnhatbui/sign-language-classification?scriptVersionId=132529947" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sign-language-mnist/sign_mnist_test.csv
/kaggle/input/sign-language-mnist/amer_sign2.png
/kaggle/input/sign-language-mnist/amer_sign3.png
/kaggle/input/sign-language-mnist/sign_mnist_train.csv
/kaggle/input/sign-language-mnist/american_sign_language.PNG
/kaggle/input/sign-language-mnist/sign_mnist_test/sign_mnist_test.csv
/kaggle/input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv


In [2]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.utils import np_utils
from sklearn.metrics import accuracy_score

In [3]:
mnist_df = pd.read_csv("/kaggle/input/sign-language-mnist/sign_mnist_train.csv")

In [4]:
train = mnist_df.iloc[:, 1:].values
y_label = mnist_df.iloc[:, 0].values

In [5]:
x_train, x_val, y_train, y_val = train_test_split(train, y_label, train_size=0.8)

In [6]:
x_train = x_train / 255

In [7]:
x_val = x_val / 255

In [8]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_val = x_val.reshape(x_val.shape[0], 28, 28, 1)

In [9]:
y_train = np_utils.to_categorical(y_train, 25)
y_val = np_utils.to_categorical(y_val, 25)

In [10]:
generator = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=20,
                                                            zoom_range=0.10,
                                                            width_shift_range=0.5,
                                                            height_shift_range=0.5,
                                                            shear_range=0.1,
                                                            horizontal_flip=True,
                                                            vertical_flip=True,
                                                            fill_mode="nearest")

In [11]:
X_train = generator.flow(x_train, y_train, batch_size=32)
X_val = generator.flow(x_val, y_val, batch_size=32)

In [12]:
X_train

In [13]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(50, 5, activation="relu", input_shape=(28,28,1)),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(50, 3, activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(25, activation="softmax")
])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 50)        1300      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 50)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 10, 10, 50)        22550     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 50)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1250)              0         
                                                                 
 dense (Dense)               (None, 25)                3

In [15]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [16]:
model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=["accuracy"])

In [17]:
x_train.shape, y_train.shape, x_val.shape, y_val.shape

((21964, 28, 28, 1), (21964, 25), (5491, 28, 28, 1), (5491, 25))

In [18]:
model.fit(
    X_train,
    epochs=10,
    validation_data=X_val,
    verbose=2
)

Epoch 1/10
687/687 - 18s - loss: 3.1661 - accuracy: 0.0547 - val_loss: 3.1255 - val_accuracy: 0.0692 - 18s/epoch - 26ms/step
Epoch 2/10
687/687 - 11s - loss: 3.0872 - accuracy: 0.0819 - val_loss: 3.0012 - val_accuracy: 0.1080 - 11s/epoch - 16ms/step
Epoch 3/10
687/687 - 10s - loss: 2.9157 - accuracy: 0.1176 - val_loss: 2.8206 - val_accuracy: 0.1390 - 10s/epoch - 15ms/step
Epoch 4/10
687/687 - 11s - loss: 2.7963 - accuracy: 0.1428 - val_loss: 2.7588 - val_accuracy: 0.1517 - 11s/epoch - 16ms/step
Epoch 5/10
687/687 - 11s - loss: 2.7365 - accuracy: 0.1561 - val_loss: 2.6761 - val_accuracy: 0.1757 - 11s/epoch - 15ms/step
Epoch 6/10
687/687 - 10s - loss: 2.6670 - accuracy: 0.1732 - val_loss: 2.6247 - val_accuracy: 0.1874 - 10s/epoch - 15ms/step
Epoch 7/10
687/687 - 11s - loss: 2.6292 - accuracy: 0.1917 - val_loss: 2.5668 - val_accuracy: 0.2076 - 11s/epoch - 16ms/step
Epoch 8/10
687/687 - 10s - loss: 2.5626 - accuracy: 0.2049 - val_loss: 2.5465 - val_accuracy: 0.2076 - 10s/epoch - 15ms/step


In [19]:
y_pred = model.predict(x_val)

172/172 [==============================] - 0s 2ms/step


In [20]:
y_pred2 = np.array([np.argmax(x) for x in y_pred])
y_val2 = np.array([np.argmax(x) for x in y_val])

In [21]:
mnist_test = pd.read_csv("/kaggle/input/sign-language-mnist/sign_mnist_test/sign_mnist_test.csv")
mnist_test.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,6,149,149,150,150,150,151,151,150,151,...,138,148,127,89,82,96,106,112,120,107
1,5,126,128,131,132,133,134,135,135,136,...,47,104,194,183,186,184,184,184,182,180
2,10,85,88,92,96,105,123,135,143,147,...,68,166,242,227,230,227,226,225,224,222
3,0,203,205,207,206,207,209,210,209,210,...,154,248,247,248,253,236,230,240,253,255
4,3,188,191,193,195,199,201,202,203,203,...,26,40,64,48,29,46,49,46,46,53


In [22]:
x_val.shape

(5491, 28, 28, 1)

In [23]:
test = mnist_test.iloc[:, 1:].values.reshape(-1, 28, 28, 1)
y_test = mnist_test.iloc[:, 0].values

In [24]:
y_pred_test = model.predict(test)

225/225 [==============================] - 0s 2ms/step


In [25]:
y_pred_test = np.array([np.argmax(x) for x in y_pred_test])

In [26]:
accuracy_score(y_pred_test, y_test)

0.0790574456218628

In [27]:
y_pred2

array([ 8, 14, 24, ...,  0, 18,  0])